# Functional programming in Scala



# What is functional programming?

Pure functional programming has a few base concepts

* Immutable values
* First class functions
* Referential transparency
* Lazy evaluation

## First class functions

From Wikipedia:
> This means the language supports passing functions as arguments to other functions, returning them as the values from other functions, and assigning them to variables or storing them in data structures


In [23]:
//Assign a function to a variable
val fn = (name: String) => s"Hello $name"
fn("Bob")

//Return a function from another function
def getFn() = (name: String) => s"Hello $name"
getFn()("Bob")

//Passing functions as arguments
def greet(fn: String => String, name: String) = {
    fn(name)
}
greet(fn, "Bob")

// Storing functions in data structures
List(fn).head("Bob")

fn: String => String = ammonite.$sess.cmd22$Helper$$Lambda$3081/565728303@16fb636f
res22_1: String = "Hello Bob"
defined function getFn
res22_3: String = "Hello Bob"
defined function greet
res22_5: String = "Hello Bob"
res22_6: String = "Hello Bob"

## Referential transparency

From Wikipedia:
> The function `rt` is referentially transparent, which means that if `x == y` then `rt(x) == rt(y)`

It is a function whose output depends entirely on its input values and has no side effects. If a function is referentially transparent, it is a pure function. 


In [22]:
// This is pure
def greet(name: String) = {
    s"Hello $name"
}
greet("Bob") == greet("Bob")


defined function greet
res21_1: Boolean = true

In [21]:
import java.lang.Math
var index = 0
val greetings = List("Hello", "Goodbye")

// This is not pure as it depends on a value outside the function which can change
def greet(name: String) = {
    index = if(index == 0) 1 else 0
    s"${greetings(index)} $name"
    
}
greet("Bob") == greet("Bob")

import java.lang.Math

index: Int = 0
greetings: List[String] = List("Hello", "Goodbye")
defined function greet
res20_4: Boolean = false

In [20]:
def writeToDatabase(greeting: String) = {
  Math.round(Math.random()).toInt match {
    case 0 => "OK"
    case 1 => "Error"
  }
}

// This is not pure as it performs IO which can return different results on different function calls.
def greet(name: String) = {
  writeToDatabase(s"Hello $name")  
}

greet("Bob") == greet("Bob")

defined function writeToDatabase
defined function greet
res19_2: Boolean = true

## Lazy evaluation

Lazy evaluation means that an expression isn't evaluated until it is first used. This is often used in combination with memoisation, which caches the result of the expression to speed up subsequent calls.


In [19]:
import java.lang.Math 
//def is lazy and not memoised
def lazyDef = Math.random

//lazy val is lazy and memoised
lazy val lazyVal = Math.random

//val is eager and memoised
val eagerVal = Math.random //Math.random is evaluated here

//Calling lazyDef twice will produce different values
lazyDef //Math.random is evaluated here
lazyDef //And here

//Calling lazyVal twice produces the same value
lazyVal //Math.random is evaluated here
lazyVal //But not here

//Calling eagerVal 
eagerVal //Math.random is not evaluated here
eagerVal //Or here

import java.lang.Math 
//def is lazy and not memoised

defined function lazyDef
lazyVal: Double = [lazy]
eagerVal: Double = 0.5855118785969868
res18_4: Double = 0.5907506066659763
res18_5: Double = 0.022403176392656654
res18_6: Double = 0.8125512485810044
res18_7: Double = 0.8125512485810044
res18_8: Double = 0.5855118785969868
res18_9: Double = 0.5855118785969868

## The problem

Functional languages like Haskell have never been at the top of the popularity lists and probably never will be. While languages like Java and Python have been incorporating functional features into the language for some time, many people never use them and never want to. What stops people writing programs in a pure functional style. I don't know but there are some possibilities.

* Many people are taught with Java, Python or Javascript and these are still [some of the most popular languages](https://insights.stackoverflow.com/survey/2020#technology-programming-scripting-and-markup-languages) These were never designed as functional languages and so they're not taught as such.
* The method of teaching functional programming is often not friendly to beginners. Its history starts with lambda calculus and modern languages use a lot of terminology from category theory. A lot of teaching materials still start with this maths heavy approach which is difficult and most don't start with "Hello World" as this is IO and is usually avoided where possible.

## Why bother then?

It is true that people can write clean, correct, readable programs using purely imperative code. So what advantages do we get from using pure functions.

* It is modular. Each function is self contained and lazily evaluated so you are always sure that you will get the same output from the same input. You can then glue these modules together into a larger program. 
* You can swap out the implementation of any of the modules for a different one providing the method contract remains the same.
* Each individual module can be tested in isolation.
* Because state is never modified directly, it can be useful for building distributed systems.


## Side effects

> A side effect is when a function relies on, or modifies, something outside its parameters to do something

This includes calls to a database, requests to an external API, receiving messages from an external queue, in short, almost everything that makes a program do useful work. So completely pure programs are rarely useful on their own and some amount of IO is needed. I'll focus on one way of doing this using the [Cats library for Scala](https://typelevel.org/cats/)

## Monads

Monads need a quick mention here. There is a lot of very maths heavy literature out there about monads which we don't need to worry about. A monad in Scala is an object which wraps another object and has a flatMap function. There are many built in ones. `Either` and `Option` being two of the most common.

In [18]:
val eitherOk: Either[String, String] = Right("OK")
// Use flat map to chain Either
eitherOk.flatMap(e => {
    Right(s"Result is $e")
})

// Or use a for comprehension which is equivalent
for {
    a <- eitherOk
    b <- Right(s"Result is $a")
} yield b

// Works for Option
for {
    a <- Some("Lots")
    b <- Some("of")
    c <- Some("options")
} yield s"$a $b $c"


eitherOk: Either[String, String] = Right("OK")
res17_1: Either[String, String] = Right("Result is OK")
res17_2: Either[String, String] = Right("Result is OK")
res17_3: Option[String] = Some("Lots of options")

In [17]:
// You can't mix them though
for {
    a <- Some("Lots")
    b <- Right("Either")
    c <- Some("options")
} yield s"$a $b $c"

cmd17.sc:4: type mismatch;
 found   : Option[String]
 required: scala.util.Either[?,?]
    c <- Some("options")
      ^cmd17.sc:3: type mismatch;
 found   : scala.util.Either[Nothing,Nothing]
 required: Option[?]
    b <- Right("Either")
      ^Compilation Failed

: 

## The end of the world

A common solution to the problem of trying to carry out interesting side effects in what is supposed to be a purely functional program is to wrap the code which carries out the side effect in a set of monads which are lazily evaluated. The different monads are chained together with flatMap and are only executed "at the end of the world" i.e. at the very end of the program. With the cats library, this monad is called `IO`

This is the first few lines of the `IO` object. The apply method takes the body and calls delay on it. This is then executed at the end of the program, either by calling IOs `unsafeRunSync` method directly or by using helper classes like `IOApp` from the `cats-effect` library.



In [ ]:
object IO extends IOInstances {

  /**
   * Suspends a synchronous side effect in `IO`.
   *
   * Alias for `IO.delay(body)`.
   */
  def apply[A](body: => A): IO[A] =
    delay(body)
    
  //Lots more stuff
}

## Is my function pure now?

Sort of...

In [ ]:
import $ivy.`org.typelevel::cats-effect:2.2.0`
import scala.concurrent.{Future, Await}
import scala.concurrent.duration._
import cats.effect._
import scala.util.Try
import java.io.{File, FileOutputStream}

In [17]:
var index = 0
def writeToDatabase(greeting: String) = {
  index = index + 1
  if(index % 2 == 0) "OK" else "Error"
}
val io = IO(writeToDatabase("Bob"))
index //writeToDatabase has not run

io.unsafeRunSync()
index //writeToDatabase has run

index: Int = 1
defined function writeToDatabase
io: IO[String] = Delay(
  ammonite.$sess.cmd16$Helper$$Lambda$2928/1101382858@3632f0c0
)
res16_3: Int = 0
res16_4: String = "Error"
res16_5: Int = 1

Calling `writeToDatabase` within the `IO` monad no longer runs the code inside the function. So if you repeatedly call `IO(writeToDatabase("Bob"))`, you will always get an `IO` wrapper around the function code. This does technically meet the definition of a pure function but on it's own, it isn't that useful until you run the `IO` wrapped code at the end of the world.

Martin Odersky said
> The IO monad does not make a function pure. It just makes it obvious that it’s impure.

Which is a reasonable reason for using it anyway. There are other advantages though.

## Are Scala Futures in the past?

The next snippet shows three features of `Future` that may cause problems.
* The first shows that `Future` is eager. The statement `Future(println("Future is eager"))` executes the side effect but the result is lost as it is not assigned to a variable. You can also see that the final statement started before the second statement.
* The second shows that `Future` is memoised. This breaks referential transparency. You can't replace the value `a` with the body `Future(println("Future is memoised")` This may become a problem when refactoring existing `Future` code.
* The ExecutionContext needs to be in every class where you're carrying out any operations on `Future`. It needs to be passed implicitly down through many layers of classes inside an application

`IO` in comparison is lazily evaluated and not memoised. And seeing as `IO` is just an object with an `apply` method, you don't need an `ExecutionContext` every time you carry out an operation on them.

In [16]:
//ExecutionContext gets everywhere
implicit val ec: scala.concurrent.ExecutionContext = scala.concurrent.ExecutionContext.global
Future(println("Future is eager")) //msg has executed even though Future(msg) isn't assigned to anything and the result is lost

val a = Future(println("Future is memoised"))
Future.sequence(List(a, a, a, a)) // Only prints once

// Prints four times
Future.sequence(List(Future(println("Print 4 times")),Future(println("Print 4 times")), Future(println("Print 4 times")), Future(println("Print 4 times"))))


Future is eager
Print 4 times
Future is memoised
Print 4 times
Print 4 times
Print 4 times


ec: scala.concurrent.ExecutionContext = scala.concurrent.impl.ExecutionContextImpl$$anon$3@7c977c09[Running, parallelism = 12, size = 7, active = 0, running = 0, steals = 96, tasks = 0, submissions = 0]
res15_1: Future[Unit] = Success(())
a: Future[Unit] = Success(())
res15_3: Future[List[Unit]] = Success(List((), (), (), ()))
res15_4: Future[List[Unit]] = Success(List((), (), (), ()))

## One monad to rule them all

There are often methods which return `Either` methods which return `Option` and methods which return `Try` and trying to get all of the different values out to process them can result in some convoluted method signatures like the `Either[String, Option[Try[String]]]` below. 

`IO` has methods to convert the existing Scala monads into instances of IO which then allows you to construct a program using a for comprehension which, if done properly, can make the flow of the program clear.

There are methods on `Option` `Either` and `Try` to convert them to the others but this along with the other IO advantages makes sense

In [24]:
def either: Either[Throwable, String] = Right("Either")
def option = Some("Option")
def trying = Try("Try")

either.map(e => {
    option.map(o => {
        trying.map(t => {
            s"$e $o $t"
        })
    })
})

(for {
    e <- IO.fromEither(either)
    o <- IO.fromOption(option)(new Exception("No option"))
    t <- IO.fromTry(trying)
} yield s"$e $o $t").unsafeRunSync()


defined function either
defined function option
defined function trying
res23_3: Either[Throwable, Option[Try[String]]] = Right(
  Some(Success("Either Option Try"))
)
res23_4: String = "Either Option Try"

## Resource safety

The `cats-effect` library provides a way of safely handling resources. Using `Resource` allows you to allocate and release resources like input and output streams safely, even if there is a cancellation or an exception. This can be done without `cats-effect` of course but `Resource` makes it clear and concise.

In [ ]:
val stream = IO(new java.io.FileOutputStream(new File("")))
Resource.make(stream)(str => {
    IO(str.close())    
}).use(stream => {
    IO(stream.write("".getBytes))
})

## Other features
There are several other useful features that we aren't yet using but could be useful.

* [Fibers](https://typelevel.org/cats-effect/docs/tutorial#intro-to-fibers) Lightweight, thread like processes which can be used for concurrent programming.
* [Tracing](https://typelevel.org/cats-effect/docs/2.x/guides/tracing) As `IO` is asynchronous, it can be difficult to work out from the stack trace what is happening when there are errors. The tracing feature allows you to view the execution details of a `cats-effect` fiber.

## Summary
* Functional programming is built around the concepts of referential transparency and lazy evaluation
* `IO` is an monad object which wraps other Scala code and delays execution.
* This object can be composed using `flatMap` into larger programs.